In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utility import save_numpy_to_h5_dataset, load_h5_dataset
import numpy as np
from matplotlib import pyplot as plt
plt.ion()   # interactive mode

Classification model with simple fully connected model. No success with this model (convergence on training set easily obtained, for validation set basically random results)

In [47]:
class simple_dense(nn.Module):
    def __init__(self):
        super(simple_dense, self).__init__()
        
        self.fc1 = nn.Linear(39*30, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 2) 

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc2(x))
        F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc3(x))
        return x

In [3]:
CUDA = True
BATCH_SIZE = 32

In [4]:
X = load_h5_dataset('X.h5')
Y = load_h5_dataset('Y.h5')

In [5]:
print (X.shape)

(1034, 39, 60)


In [6]:
X = X.astype('float32')

In [7]:
X = X[:,:,:30]

In [9]:
X = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))

In [10]:
X.shape

(1034, 1170)

In [11]:
np.random.seed(100) #seed fixed for reproducibility
mask = np.random.rand(len(X)) < 0.9  #array of boolean variables

training_images = X[mask]
training_labels = Y[mask]

validation_images = X[~mask]
validation_labels = Y[~mask]

training_images = torch.from_numpy(training_images) #convert to torch tensor
training_labels = torch.from_numpy(training_labels) #convert to torch tensor

validation_images = torch.from_numpy(validation_images) #convert to torch tensor
validation_labels = torch.from_numpy(validation_labels) #convert to torch tensor

In [12]:
training_labels = training_labels.long()
validation_labels = validation_labels.long()

In [13]:
train_dataset = torch.utils.data.TensorDataset(training_images, training_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

validation_dataset = torch.utils.data.TensorDataset(validation_images, validation_labels)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [48]:
net = simple_dense()

In [49]:
criterion = nn.CrossEntropyLoss()
net.cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

In [50]:
EPOCHS = 20

In [52]:
# TRAINING #


for epoch in range(EPOCHS):  
    net.train()
    running_loss = 0.0
    for j, data in enumerate(train_loader, 0):
            # get the inputs
            inputs, labels = data
            batch_length = len(inputs) #length of the current batch
            
            # wrap them in Variable
            if (CUDA):
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            output = net(inputs)
                    
            loss = criterion(output, labels)
            running_loss += loss.data[0]*batch_length
            loss.backward()
            optimizer.step()
            
    print('Epoch %d, loss: %.3f' % (epoch + 1, running_loss / len(training_images)))     
    
    
    #VALIDATION
    net.eval()
    running_loss = 0.0
    for j, data in enumerate(validation_loader, 0):
            # get the inputs
            inputs, labels = data
            batch_length = len(inputs) #length of the current batch
            
            # wrap them in Variable
            if (CUDA):
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            # forward
            output = net(inputs)
                    
            loss = criterion(output, labels)
            running_loss += loss.data[0]*batch_length
            
    print('Epoch %d, validation loss: %.3f' % (epoch + 1, running_loss / len(validation_images)))     
    
    
    
    

Epoch 1, loss: 0.613
Epoch 1, validation loss: 0.726
Epoch 2, loss: 0.609
Epoch 2, validation loss: 0.715
Epoch 3, loss: 0.606
Epoch 3, validation loss: 0.713
Epoch 4, loss: 0.604
Epoch 4, validation loss: 0.719
Epoch 5, loss: 0.600
Epoch 5, validation loss: 0.726
Epoch 6, loss: 0.593
Epoch 6, validation loss: 0.718
Epoch 7, loss: 0.591
Epoch 7, validation loss: 0.723
Epoch 8, loss: 0.586
Epoch 8, validation loss: 0.726
Epoch 9, loss: 0.581
Epoch 9, validation loss: 0.727
Epoch 10, loss: 0.578
Epoch 10, validation loss: 0.741
Epoch 11, loss: 0.575
Epoch 11, validation loss: 0.738
Epoch 12, loss: 0.569
Epoch 12, validation loss: 0.739
Epoch 13, loss: 0.565
Epoch 13, validation loss: 0.737
Epoch 14, loss: 0.566
Epoch 14, validation loss: 0.736
Epoch 15, loss: 0.558
Epoch 15, validation loss: 0.739
Epoch 16, loss: 0.557
Epoch 16, validation loss: 0.739
Epoch 17, loss: 0.551
Epoch 17, validation loss: 0.745
Epoch 18, loss: 0.547
Epoch 18, validation loss: 0.751
Epoch 19, loss: 0.545
Epoch 